In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_path="../../models/hf/1_step30800"
#model_path= "llm-jp/llm-jp-13b-v1.0"

model_dict={

#"1_step30800":"../../models/hf/1_step30800",
#"2_step37800":"../../models/hf/2_step37800",
#"3_step43400":"../../models/hf/3_step43400",
#"4_step49700":"../../models/hf/4_step49700",
#"5_step58800":"../../models/hf/5_step58800",
"epoch3_step300":"../../models/hf/epoch3_step300",
"epoch2_step20644":"../../models/hf/epoch2_step20644",
"llmjp13b":"llm-jp/llm-jp-13b-v1.0",
}


In [3]:

def init_pipe(model_name):
    model_path=model_dict[model_name]
    model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
                max_new_tokens=100, 
                repetition_penalty=1.2,
                temperature=0.7,
                )
    return model,tokenizer,pipe

In [4]:
import time

text_list=[
"富士山は",
"化学研究の問題点は",
"1+2+3-4=",
"Functional polymers are",
"Bioinformatics is",
"フィボナッチ数列を生成するPythonのコードは次のとおりです｡\n def",
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
"英語: What is the problem of developing LLM in Japan? 日本語: ",
"Q: 日本の首都は? A:",
"Q: ドラえもんの友達は? A:",
"Q: 純粋理性批判はたぬきに理解できますか? A:",
"Q: ChatGPTとはなんですか? A:",
"Q: 俳句を書いてください A:",
"Q: 日本の総理大臣は? A:",
]

out_dict={}

for model_name in model_dict.keys():
    print(model_name)
    model,tokenizer,pipe=init_pipe(model_name)
    model_output_dict={}
    for text in text_list:
        perp=perplexity(model,tokenizer,text)
        s_time=time.time()
        res=pipe(text)[0]["generated_text"]
        consumed_time=time.time()-s_time
        model_output_dict[text]={
            "perplexity":perp,
            "output":res
        }
        print("-------")
        print("input: ", text.replace("\n",""))
        print("perplexity: ",perp)
        print("time: ", consumed_time)
        print("time/character: ", consumed_time/len(res))
        print("output: ",res.replace("\n",""))
    out_dict[model_name]=model_output_dict

epoch3_step300


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.07s/it]
/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


-------
input:  富士山は
perplexity:  703.7090454101562
time:  3.826026201248169
time/character:  0.02068122270944956
output:  富士山は、日本の象徴として世界的に有名です。 「私がこの仕事を始めたきっかけも、『人と話す事』でした」「今回ご縁があってお手伝いさせて頂くことになったのですが...」『自分自身で考えて行動する力をつける為のお勉強会なんです♪』※当日キャンセルされた場合でも参加費のご返金はできませんことを予めご了承くださいますようよろしくお願いいたしますm(__)mm(__)m(
-------
input:  化学研究の問題点は
perplexity:  99.94461059570312
time:  2.8838627338409424
time/character:  0.015020118405421575
output:  化学研究の問題点は、その成果が社会に還元されないまま終わってしまうことです。私たちは「科学と技術」を区別し、「科学技術政策大綱」(平成13年6月29日閣議決定)においてもこの考え方に立って施策展開に当たっています。「Science(サイエンス)」という言葉には自然現象や人間活動などに関する知識・理解といった意味合いがあります。一方で同じく英語圏であるアメリカ合衆国では日本国内以上
-------
input:  1+2+3-4=
perplexity:  17.39277458190918
time:  2.8768863677978516
time/character:  0.009018452563629629
output:  1+2+3-4=0        self.assertEqual(len([x for x in self._data_list() if not (not isinstance(type, str) and type =='string' or \                    issubclass('dict', Type))])) # -*- coding: utf8; tabindexing: false; namedbcrow::tabs->tablenameb

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


-------
input:  Q: 日本の首都は? A:
perplexity:  30.43387222290039
time:  2.8490967750549316
time/character:  0.01325161290723224
output:  Q: 日本の首都は? A: トウキョウ B: 東京 C: 大阪 D: 京都 E: 横浜 F: 名古屋 G: 福岡 H: 札幌 I: 仙台 J: 神戸 K: 広島 L: 岡山 M: 奈良 N: 三重 O: 滋賀 P: 岐阜 S: 長野 T: 富山 U: 紫波 V: 岩手 W: 青森 X: 秋田 Y: 山形 Z: 福島 #!/usr/bin/env python# -*- coding: utf-8; py27py36 -sdk
-------
input:  Q: ドラえもんの友達は? A:
perplexity:  30.147478103637695
time:  2.8688032627105713
time/character:  0.011614588108139964
output:  Q: ドラえもんの友達は? A:ジャイアン B:スネ夫 C:しずか D:出木杉Q.「10」を英語で言うと何になるでしょうか。A:「テン(ten)」です。「十」(じゅう)ではありませんよ! # -*- coding: utf-8; mode: python -m venv/bin_venv2649375e\n# Copyright (c) Microsoft Corporation, All rights reserved. \usepackage{amsmath} % This is a
-------
input:  Q: 純粋理性批判はたぬきに理解できますか? A:
perplexity:  33.83030700683594
time:  2.8812665939331055
time/character:  0.012208756753953836
output:  Q: 純粋理性批判はたぬきに理解できますか? A:無理です。 「私の名前を知っているということは、あなたもこの世界から来た人ですね」「ええ......まあ......」俺が曖昧な返事をしている間に彼女――いや彼と言うべきだろうか――の頭の上に

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.14s/it]


-------
input:  富士山は
perplexity:  479.4016418457031
time:  3.566544532775879
time/character:  0.017656161053345935
output:  富士山は、日本の象徴であり日本人にとって特別な山です。 「私がこの仕事を始めたきっかけも『人に喜んでもらいたい』という思いからでした」と語る代表取締役社長・鈴木孝幸さん。「自分自身で何か事業をはじめたい」「社会貢献できるような活動ができれば...」そんな想いがある方にはぜひチャレンジしてほしい会社だと感じました!「当社では、『ありがとう』『助かったよ!』など感謝される機会がとても多いんです!」そう話
-------
input:  化学研究の問題点は
perplexity:  88.73136138916016
time:  3.0521726608276367
time/character:  0.014673907023209792
output:  化学研究の問題点は、その成果が社会に還元されないままになっていることである。このような状況を打破するためには、「科学者」と「市民・国民」、「企業人(技術系)」などの間で情報や意見交換ができる場をつくることが必要であり、そのためにも大学等においても積極的な取り組みが求められるところから本事業では次のような活動をしている: 「私たちは今回初めて参加させていただきました!普段なかなか体験できないことをたくさん経験できて
-------
input:  1+2+3-4=
perplexity:  14.325879096984863
time:  2.847233772277832
time/character:  0.014526702919784857
output:  1+2+3-4=7となります。このように、数式を使って計算すると簡単に答えが出ますし、「なぜそうなるのか」という理由も分かりやすく説明できますね! 「私は今までの人生で一度たりとも自分のことを『ブス』だと思ったことはありません!」(原文ママ)ーそう語るのはタレント・モデルとして活躍する“みちょぱ"こと池田美優さんだ。「コンプレックスだらけだった」「自信がなかった頃から変わるために努力した
-------
in

Loading checkpoint shards: 100%|██████████| 3/3 [00:44<00:00, 14.92s/it]


-------
input:  富士山は
perplexity:  1988.0616455078125
time:  1.2500052452087402
time/character:  0.02403856240786039
output:  富士山は、日本の象徴として古くから親しまれてきました。その美しい姿をいつまでも後世に伝えたいと思います。
-------
input:  化学研究の問題点は
perplexity:  191.06161499023438
time:  0.8797569274902344
time/character:  0.025135912214006698
output:  化学研究の問題点は、「科学的」な方法論が確立されていないことである。
-------
input:  1+2+3-4=
perplexity:  19.304046630859375
time:  4.190646648406982
time/character:  0.002767930415064057
output:  1+2+3-4=0    /*!< The number of bits in the data field. This is always a power of two and must be equal to or greater than zero, otherwise an error will occur when reading from this register.                                                                                                                                                                                                                                                                                                                                                

In [8]:
import json
#with open("test/output.json","w") as f:
#    json.dump(out_dict,f,indent=4,ensure_ascii=False)

In [9]:
import pandas as pd
df=pd.DataFrame(out_dict)

perp_dict={}
text_dict={}
for model_name in model_dict.keys():
    temp_dict={}
    temp_dict2={}
    for text in text_list:
        temp_dict[text]=out_dict[model_name][text]["perplexity"]
        temp_dict2[text]=out_dict[model_name][text]["output"][len(text):]
    perp_dict[model_name]=temp_dict
    text_dict[model_name]=temp_dict2

df_perp=pd.DataFrame(perp_dict)
df_perp

,epoch3_step300,epoch2_step20644,llmjp13b
富士山は,703.709045,479.401642,1988.061646
化学研究の問題点は,99.944611,88.731361,191.061615
1+2+3-4=,17.392775,14.325879,19.304047
Functional polymers are,77.874954,100.534294,36.150234
Bioinformatics is,16.166267,16.905458,13.367496
フィボナッチ数列を生成するPythonのコードは次のとおりです｡\n def,43.573914,52.979744,20.442436
ホンダ フィット 販売,625.540955,832.587158,117.768806
英語: He is a good man. 日本語:,28.333467,24.340479,21.825132
英語: What is the problem of developing LLM in Japan? 日本語:,47.022423,43.724861,42.456501
Q: 日本の首都は? A:,30.433872,28.396450,29.827217


In [10]:
df_text=pd.DataFrame(text_dict)
df_text

,epoch3_step300,epoch2_step20644,llmjp13b
富士山は,、日本の象徴として世界的に有名です。 「私がこの仕事を始めたきっかけも、『人と話す事』でした...,、日本の象徴であり日本人にとって特別な山です。 「私がこの仕事を始めたきっかけも『人に喜んで...,、日本の象徴として古くから親しまれてきました。その美しい姿をいつまでも後世に伝えたいと思います。
化学研究の問題点は,、その成果が社会に還元されないまま終わってしまうことです。\n私たちは「科学と技術」を区別し...,、その成果が社会に還元されないままになっていることである。このような状況を打破するためには、...,、「科学的」な方法論が確立されていないことである。\n
1+2+3-4=,0\n self.assertEqual(len([x for x in se...,7\nとなります。このように、数式を使って計算すると簡単に答えが出ますし、「なぜそうなるのか...,0\n /*!< The number of bits in the data fie...
Functional polymers are,"widely used in the field of medicine, such as...","widely used in the food, pharmaceutical and c...","used in a wide variety of applications, inclu..."
Bioinformatics is,a field of science that uses computers to ana...,a field of science that uses computers to ana...,a bimonthly peer-reviewed scientific journal ...
フィボナッチ数列を生成するPythonのコードは次のとおりです｡\n def,get_data(self): #TODO: this is already in the...,get_next(self): return self.get() +1 「お前が、俺に何...,fib(n): # n番目までの和が1になるような自然数。\n if n<2 or...
ホンダ フィット 販売,台数1位の車種です。\n20代女性が選ぶ、おすすめ人気ブランド腕時計をまとめました! 「私は...,終了\n2017年9月に生産を終えた「N-BOX」の後継モデルとして、新型車が登場する。その...,店：オートショップ タキーズ 年式(初度登録) 2013(平成25)年 走行距離 4.9万k...
英語: He is a good man. 日本語:,あなたはいい人です。 「私の場合、10年以上前に受けた手術が原因で発症しました」と話す女性患...,あいつはいい奴だ。 「お前が、俺の事を好きだと言ってくれたから......だから」\n「うん...,彼は良い人です。\nHe's not bad at all!（全く悪くないよ！）：「全然」の...
英語: What is the problem of developing LLM in Japan? 日本語:,日本人がLMLを開発するのに問題はあるか。 #!/usr/bin/env python\n#...,LL Mの発展問題は何ですか。 「私が、この子を産んだ時もそうだったわ」\n「え?」と僕ら三...,日本におけるLL.Mの発展について問題となることは何ですか？\n 英語: How do yo...
Q: 日本の首都は? A:,トウキョウ B: 東京 C: 大阪 D: 京都 E: 横浜 F: 名古屋 G: 福岡 H:...,トーキョー B: 東京 C: Tokyo D: Tōkyū E: Tokio F: Kyō...,東京です。Q. 日本で一番高い山は？A：富士山です。Q．日本には何があるか知っている人います...
